### 01. Importar Bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib
from sklearn.metrics import accuracy_score


### 02. Abrir arquivos

In [1]:
df = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

### 03. Transformar a coluna `nota_mat` em log

In [1]:
df['nota_mat'] = np.log(df['nota_mat'])

### 04. Juntar os 2 DataFrames em uma única variável

In [1]:
df = df.append(test, sort=False)

In [1]:
df.shape

### 05. Transformar texto em número, exceto a codigo_num

In [1]:
for c in df.columns:
    if (df[c].dtype == 'object') & (c != 'codigo_mun'):
        df[c] = df[c].astype('category').cat.codes

In [1]:
#Retirando a string ID_ID_ da Coluna
df['codigo_mun'] = df['codigo_mun'].str.replace('ID_ID_','')

### 06. Preencher Valores em Branco

In [1]:
# -2 para a separação posterior das bases
df['nota_mat'].fillna(-2, inplace=True)

In [1]:
df.fillna(0, inplace=True)

### 07. Separar as bases de volta em `test` e `df`



In [1]:
df, test = df[df['nota_mat']!=-2], df[df['nota_mat']==-2]

### 08. Criar os conjuntos de treino e validação


In [1]:
train, valid = train_test_split(df, random_state=42)

### 09. Treinar o modelo


In [1]:
rf = RandomForestRegressor(random_state=42, n_estimators=100)

In [1]:
feats = [c for c in df.columns if c not in ['nota_mat']]

In [1]:
rf.fit(train[feats], train['nota_mat'])

### 10. Fazer previsões

In [1]:
valid_preds = rf.predict(valid[feats])

In [1]:
mean_squared_error(valid['nota_mat'], valid_preds)**(1/2)

In [1]:
mean_squared_error(valid['nota_mat'], valid_preds)**(1/2)

### Enviar pro Kaggle

In [1]:
test['nota_mat'] = np.exp(rf.predict(test[feats]))

In [1]:
test[['codigo_mun','nota_mat']].to_csv('rf1.csv', index=False)